<a href="https://colab.research.google.com/github/TheClassicTechno/cleansea_model/blob/main/another_version/small_taco_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pathlib
import cv2
import glob

#processing images
bottles1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/bottles'
cans1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/cans'
containers1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/containers'
paper1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/paper'
plastic1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/plastic'

#all images are 227x227 in RGB so 227, 227, 3
bottles = [cv2.imread(image) for image in glob.glob(bottles1)]
cans = [cv2.imread(image) for image in glob.glob(cans1)]
containers = [cv2.imread(image) for image in glob.glob(containers1)]
paper = [cv2.imread(image) for image in glob.glob(paper1)]
plastic = [cv2.imread(image) for image in glob.glob(plastic1)]

train_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/TEAM9SURESTART/taco',
    validation_split = 0.2,
    subset = "training",
    seed = 24,  
    image_size = (227, 227),
    batch_size = 8
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/TEAM9SURESTART/taco',
    validation_split = 0.2,
    subset = "validation",
    seed = 24,  
    image_size = (227, 227),
    batch_size = 8
)


from scipy.stats import zscore
import pandas as pd

# Create a sample df

# Calculate the zscores and drop zscores into new column

train_dataset = train_dataset.cache().shuffle(13).prefetch(buffer_size = tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(13).prefetch(buffer_size = tf.data.AUTOTUNE)

model = Sequential ([
    layers.Rescaling(1./255, input_shape = (227, 227, 3)),
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(5, activation = 'softmax')
])
model.summary()


Found 172 files belonging to 5 classes.
Using 138 files for training.
Found 172 files belonging to 5 classes.
Using 34 files for validation.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 227, 227, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 227, 227, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 113, 113, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 113, 113, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                           

In [3]:
len(train_dataset)

18

In [4]:
#model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy'],  run_eagerly=True
)
model.fit (
    train_dataset, 
    validation_data = val_dataset,
    epochs = 120
)
#model.save('model.h5')

Epoch 1/120
18/18 [==============================] - 26s 196ms/step - loss: 2.1523 - accuracy: 0.2971 - val_loss: 1.5028 - val_accuracy: 0.2353
Epoch 2/120
18/18 [==============================] - 1s 32ms/step - loss: 1.5023 - accuracy: 0.3623 - val_loss: 1.4031 - val_accuracy: 0.4118
Epoch 3/120
18/18 [==============================] - 1s 33ms/step - loss: 1.2977 - accuracy: 0.4783 - val_loss: 1.3005 - val_accuracy: 0.5000
Epoch 4/120
18/18 [==============================] - 1s 31ms/step - loss: 1.1049 - accuracy: 0.6087 - val_loss: 1.4223 - val_accuracy: 0.5294
Epoch 5/120
18/18 [==============================] - 1s 32ms/step - loss: 0.8286 - accuracy: 0.7174 - val_loss: 1.2664 - val_accuracy: 0.5000
Epoch 6/120
18/18 [==============================] - 1s 31ms/step - loss: 0.5080 - accuracy: 0.8478 - val_loss: 1.5908 - val_accuracy: 0.4412
Epoch 7/120
18/18 [==============================] - 1s 31ms/step - loss: 0.4042 - accuracy: 0.8551 - val_loss: 1.5967 - val_accuracy: 0.5000
Epoc